In [1]:
import pickle
import statistics
import numpy as np
import pandas as pd
from datetime import datetime
import preprocessing_functions

Load Fitbit

In [3]:
fitbit = pd.read_pickle('./data/fitbit_to_baseline_model.pkl')
fitbit["date"] = pd.to_datetime(pd.to_datetime(fitbit["date"]).dt.date)
fitbit

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,Martial Arts,Run,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,621e301e67b776a240608a72,2021-05-24,34.710232,NaN,NaN,NaN,NaN,74.0,0.700000,0.700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e301367b776a24057738e,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,621e360b67b776a24039709f,2021-05-24,NaN,68.321,31.413,NaN,15.4,69.0,0.633333,0.650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,621e2fce67b776a240279baa,2021-05-24,34.783747,83.170,15.941,96.6,15.2,70.0,0.733333,0.725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959,621e345267b776a240691064,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4960,621e310d67b776a24003096d,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4961,621e324e67b776a2400191cb,2022-01-17,33.104768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4962,621e30f467b776a240f22944,2022-01-17,34.352490,NaN,NaN,NaN,13.4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Load TTM

In [4]:
ttm = pd.read_pickle('./data/surveys/ttm.pkl')
ttm = ttm.rename(columns={"user_id": "id", "submitdate": "date"})
ttm["date"] = pd.to_datetime(pd.to_datetime(ttm["date"]).dt.date)
ttm = ttm.drop_duplicates()
ttm = ttm.drop(columns=['type'])
ttm

,id,date,stage,ttm_consciousness_raising,ttm_dramatic_relief,ttm_environmental_reevaluation,ttm_self_reevaluation,ttm_social_liberation,ttm_counterconditioning,ttm_helping_relationships,ttm_reinforcement_management,ttm_self_liberation,ttm_stimulus_control
0,621e2e8e67b776a24055b564,2021-07-26,Maintenance,1.333333,2.666667,2.333333,4.333333,4.000000,4.000000,4.000000,4.000000,4.000000,1.666667
0,621e2eaf67b776a2406b14ac,2021-11-29,Contemplation,3.333333,2.666667,4.000000,4.666667,4.000000,2.333333,3.000000,4.333333,3.333333,1.000000
1,621e2eaf67b776a2406b14ac,2022-01-17,Action,3.666667,2.333333,4.333333,4.333333,3.333333,1.666667,3.000000,4.333333,3.333333,3.000000
0,621e2ed667b776a24085d8d1,2021-06-07,Preparation,1.000000,2.000000,3.000000,3.333333,3.333333,1.000000,1.000000,3.333333,1.666667,1.000000
1,621e2ed667b776a24085d8d1,2021-07-30,Contemplation,1.666667,2.333333,4.000000,3.333333,3.666667,1.333333,1.000000,2.333333,2.333333,1.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,621e36c267b776a240ba2756,2021-05-31,Action,2.666667,1.000000,3.000000,4.333333,3.666667,4.000000,3.333333,4.000000,3.666667,3.000000
0,621e36f967b776a240e5e7c9,2021-05-31,Action,2.000000,4.333333,3.333333,5.000000,3.000000,3.333333,1.666667,3.666667,4.000000,3.333333
1,621e36f967b776a240e5e7c9,2021-07-26,Action,4.000000,3.666667,3.666667,5.000000,3.000000,3.000000,1.666667,4.666667,4.000000,3.000000
0,621e375b67b776a240290cdc,2021-06-01,Action,2.333333,1.666667,1.000000,2.666667,5.000000,1.000000,3.000000,4.666667,2.666667,1.333333


Merge Fitbit with TTM

In [4]:
data = ttm.merge(fitbit, on=['date', 'id'])
data

,id,date,stage,ttm_consciousness_raising,ttm_dramatic_relief,ttm_environmental_reevaluation,ttm_self_reevaluation,ttm_social_liberation,ttm_counterconditioning,ttm_helping_relationships,...,Martial Arts,Run,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates
0,621e2e8e67b776a24055b564,2021-07-26,Maintenance,1.333333,2.666667,2.333333,4.333333,4.000000,4.000000,4.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,621e2eaf67b776a2406b14ac,2021-11-29,Contemplation,3.333333,2.666667,4.000000,4.666667,4.000000,2.333333,3.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,621e2eaf67b776a2406b14ac,2022-01-17,Action,3.666667,2.333333,4.333333,4.333333,3.333333,1.666667,3.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,621e2ed667b776a24085d8d1,2021-06-07,Preparation,1.000000,2.000000,3.000000,3.333333,3.333333,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,621e2f3967b776a240c654db,2021-05-31,Preparation,2.000000,2.000000,2.666667,4.000000,5.000000,2.333333,2.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,621e36c267b776a240ba2756,2021-05-31,Action,2.666667,1.000000,3.000000,4.333333,3.666667,4.000000,3.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,621e36f967b776a240e5e7c9,2021-05-31,Action,2.000000,4.333333,3.333333,5.000000,3.000000,3.333333,1.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
73,621e36f967b776a240e5e7c9,2021-07-26,Action,4.000000,3.666667,3.666667,5.000000,3.000000,3.000000,1.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,621e375b67b776a240290cdc,2021-06-01,Action,2.333333,1.666667,1.000000,2.666667,5.000000,1.000000,3.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
data.to_pickle('./data/fitbit_ttm_to_model.pkl')

Load BREQ

In [6]:
breq = pd.read_pickle('./data/surveys/breq.pkl')
breq = breq.rename(columns={"user_id": "id", "submitdate": "date"})
breq["date"] = pd.to_datetime(pd.to_datetime(breq["date"]).dt.date)
breq = breq.drop_duplicates()
breq = breq.drop(columns=['type'])
breq

,id,date,breq_amotivation,breq_external_regulation,breq_introjected_regulation,breq_identified_regulation,breq_intrinsic_regulation,breq_self_determination
0,621e2e8e67b776a24055b564,2021-05-31,1.00,1.00,1.000000,3.50,4.25,intrinsic_regulation
1,621e2e8e67b776a24055b564,2021-07-26,1.00,1.00,1.666667,3.50,4.00,intrinsic_regulation
0,621e2eaf67b776a2406b14ac,2021-11-29,1.25,1.00,1.666667,4.50,3.75,identified_regulation
1,621e2eaf67b776a2406b14ac,2022-01-17,1.00,1.25,2.333333,4.00,3.50,identified_regulation
0,621e2ed667b776a24085d8d1,2021-06-07,2.75,2.00,3.000000,2.50,1.00,introjected_regulation
...,...,...,...,...,...,...,...,...
0,621e36c267b776a240ba2756,2021-05-31,1.00,1.00,1.000000,3.75,4.75,intrinsic_regulation
0,621e36f967b776a240e5e7c9,2021-05-31,1.00,1.75,4.666667,4.00,3.25,introjected_regulation
1,621e36f967b776a240e5e7c9,2021-07-26,1.00,1.75,5.000000,4.75,3.25,introjected_regulation
0,621e375b67b776a240290cdc,2021-06-01,1.00,1.00,1.666667,3.50,2.75,identified_regulation


Merge Fitbit with BREQ

In [7]:
data = breq.merge(fitbit, on=['date', 'id'])
data

,id,date,breq_amotivation,breq_external_regulation,breq_introjected_regulation,breq_identified_regulation,breq_intrinsic_regulation,breq_self_determination,nightly_temperature,nremhr,...,Martial Arts,Run,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates
0,621e2e8e67b776a24055b564,2021-05-31,1.00,1.00,1.000000,3.50,4.25,intrinsic_regulation,34.476865,58.652,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,621e2e8e67b776a24055b564,2021-07-26,1.00,1.00,1.666667,3.50,4.00,intrinsic_regulation,34.212037,56.034,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,621e2eaf67b776a2406b14ac,2021-11-29,1.25,1.00,1.666667,4.50,3.75,identified_regulation,34.693016,77.985,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,621e2eaf67b776a2406b14ac,2022-01-17,1.00,1.25,2.333333,4.00,3.50,identified_regulation,34.152692,70.546,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,621e2ed667b776a24085d8d1,2021-06-07,2.75,2.00,3.000000,2.50,1.00,introjected_regulation,34.084118,79.669,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,621e36c267b776a240ba2756,2021-05-31,1.00,1.00,1.000000,3.75,4.75,intrinsic_regulation,33.094862,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,621e36f967b776a240e5e7c9,2021-05-31,1.00,1.75,4.666667,4.00,3.25,introjected_regulation,34.052594,64.918,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
73,621e36f967b776a240e5e7c9,2021-07-26,1.00,1.75,5.000000,4.75,3.25,introjected_regulation,34.354848,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,621e375b67b776a240290cdc,2021-06-01,1.00,1.00,1.666667,3.50,2.75,identified_regulation,34.863055,62.353,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [8]:
data.to_pickle('./data/fitbit_breq_to_model.pkl')

Load Personality

In [9]:
big5 = pd.read_pickle('./data/surveys/personality.pkl')
big5 = big5.rename(columns={"user_id": "id", "submitdate": "date"})
big5["date"] = pd.to_datetime(pd.to_datetime(big5["date"]).dt.date)
big5 = big5.drop_duplicates()
big5 = big5.drop(columns=['type'])
big5

,id,date,extraversion,agreeableness,conscientiousness,stability,intellect,gender,ipip_extraversion_category,ipip_agreeableness_category,ipip_conscientiousness_category,ipip_stability_category,ipip_intellect_category
0,621e2e8e67b776a24055b564,2021-05-31,21.0,33.0,45.0,42.0,40.0,MALE,LOW,LOW,HIGH,HIGH,AVERAGE
1,621e2eaf67b776a2406b14ac,2021-11-29,32.0,45.0,30.0,18.0,41.0,FEMALE,AVERAGE,HIGH,AVERAGE,LOW,HIGH
2,621e2ed667b776a24085d8d1,2021-06-07,40.0,43.0,22.0,28.0,34.0,FEMALE,HIGH,AVERAGE,LOW,AVERAGE,AVERAGE
3,621e2f3967b776a240c654db,2021-05-31,25.0,34.0,30.0,39.0,37.0,MALE,AVERAGE,AVERAGE,LOW,HIGH,AVERAGE
4,621e2f6167b776a240e082a9,2021-06-01,41.0,41.0,30.0,34.0,30.0,FEMALE,HIGH,AVERAGE,AVERAGE,HIGH,LOW
5,621e2f7a67b776a240f14425,2021-05-31,29.0,38.0,43.0,39.0,35.0,MALE,AVERAGE,AVERAGE,HIGH,HIGH,AVERAGE
6,621e2f9167b776a240011ccb,2021-11-28,21.0,45.0,31.0,18.0,39.0,FEMALE,LOW,HIGH,AVERAGE,LOW,HIGH
7,621e2fb367b776a24015accd,2021-05-31,37.0,36.0,29.0,27.0,41.0,MALE,HIGH,AVERAGE,LOW,LOW,HIGH
8,621e2fce67b776a240279baa,2021-05-31,34.0,42.0,35.0,27.0,35.0,MALE,HIGH,HIGH,AVERAGE,LOW,AVERAGE
9,621e2ff067b776a2403eb737,2021-11-29,33.0,44.0,23.0,11.0,30.0,FEMALE,AVERAGE,HIGH,LOW,LOW,LOW


Merge Fitbit with Personality

In [10]:
data = big5.merge(fitbit, on=['date', 'id'])
data

,id,date,extraversion,agreeableness,conscientiousness,stability,intellect,gender,ipip_extraversion_category,ipip_agreeableness_category,...,Martial Arts,Run,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates
0,621e2e8e67b776a24055b564,2021-05-31,21.0,33.0,45.0,42.0,40.0,MALE,LOW,LOW,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,621e2eaf67b776a2406b14ac,2021-11-29,32.0,45.0,30.0,18.0,41.0,FEMALE,AVERAGE,HIGH,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,621e2ed667b776a24085d8d1,2021-06-07,40.0,43.0,22.0,28.0,34.0,FEMALE,HIGH,AVERAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e2f3967b776a240c654db,2021-05-31,25.0,34.0,30.0,39.0,37.0,MALE,AVERAGE,AVERAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,621e2f6167b776a240e082a9,2021-06-01,41.0,41.0,30.0,34.0,30.0,FEMALE,HIGH,AVERAGE,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,621e2f7a67b776a240f14425,2021-05-31,29.0,38.0,43.0,39.0,35.0,MALE,AVERAGE,AVERAGE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,621e2f9167b776a240011ccb,2021-11-28,21.0,45.0,31.0,18.0,39.0,FEMALE,LOW,HIGH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,621e2fb367b776a24015accd,2021-05-31,37.0,36.0,29.0,27.0,41.0,MALE,HIGH,AVERAGE,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
8,621e2fce67b776a240279baa,2021-05-31,34.0,42.0,35.0,27.0,35.0,MALE,HIGH,HIGH,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
9,621e2ff067b776a2403eb737,2021-11-29,33.0,44.0,23.0,11.0,30.0,FEMALE,AVERAGE,HIGH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
data.to_pickle('./data/fitbit_personality_to_model.pkl')

Find the users

In [5]:
users = list(fitbit['id'].unique())

Load PANAS

In [6]:
panas = pd.read_pickle('./data/surveys/panas.pkl')
panas = panas.rename(columns={"user_id": "id", "submitdate": "date"})
panas["date"] = pd.to_datetime(pd.to_datetime(panas["date"]).dt.date)
panas = panas.drop_duplicates()
panas = panas.drop(columns=['type'])
panas

,id,date,positive_affect_score,negative_affect_score
1,621e2e8e67b776a24055b564,2021-07-26,37,14
2,621e2e8e67b776a24055b564,2021-05-31,38,12
3,621e2e8e67b776a24055b564,2021-06-07,37,12
4,621e2e8e67b776a24055b564,2021-06-28,31,12
5,621e2e8e67b776a24055b564,2021-07-19,37,12
...,...,...,...,...
1,621e375367b776a24021e950,2022-01-02,30,33
0,621e375b67b776a240290cdc,2021-06-01,27,29
1,621e375b67b776a240290cdc,2021-06-08,30,30
2,621e375b67b776a240290cdc,2021-06-22,30,30


In [7]:
fitbit_panas = preprocessing_functions.weekly_fitbit_frequency(panas, fitbit, users)
fitbit_panas

,id,date,positive_affect_score,negative_affect_score,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,...,Martial Arts,Run,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates
0,621e2e8e67b776a24055b564,2021-07-26,37,14,34.674633,51.192,49.232,96.2,12.2,83.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,621e2e8e67b776a24055b564,2021-05-31,38,12,34.674633,51.192,49.232,96.2,12.2,83.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,621e2e8e67b776a24055b564,2021-06-07,37,12,34.674633,51.192,49.232,96.2,12.2,83.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,621e2e8e67b776a24055b564,2021-06-28,31,12,34.674633,51.192,49.232,96.2,12.2,83.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,621e2e8e67b776a24055b564,2021-07-19,37,12,34.674633,51.192,49.232,96.2,12.2,83.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,621e339967b776a240e502de,2021-12-20,44,13,34.674633,51.192,49.232,96.2,12.2,83.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
264,621e339967b776a240e502de,2021-12-27,45,11,34.674633,51.192,49.232,96.2,12.2,83.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
265,621e339967b776a240e502de,2022-01-03,44,11,34.674633,51.192,49.232,96.2,12.2,83.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
266,621e339967b776a240e502de,2022-01-12,45,13,34.674633,51.192,49.232,96.2,12.2,83.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [8]:
fitbit_panas.to_pickle('./data/fitbit_panas_to_model.pkl')

Load STAI

In [9]:
stai = pd.read_pickle('./data/surveys/stai.pkl')
stai = stai.rename(columns={"user_id": "id", "submitdate": "date"})
stai["date"] = pd.to_datetime(pd.to_datetime(stai["date"]).dt.date)
panas = panas.drop_duplicates()
stai = stai.drop(columns=['type'])
stai

,id,date,stai_stress,stai_stress_category
0,621e2e8e67b776a24055b564,2021-05-31,45.0,Below average
1,621e2e8e67b776a24055b564,2021-06-07,46.0,Average
3,621e2e8e67b776a24055b564,2021-07-19,46.0,Average
4,621e2e8e67b776a24055b564,2021-07-26,46.0,Average
5,621e2e8e67b776a24055b564,2021-06-28,54.0,Above average
...,...,...,...,...
0,621e375b67b776a240290cdc,2021-06-08,60.0,Above average
1,621e375b67b776a240290cdc,2021-07-06,60.0,Above average
2,621e375b67b776a240290cdc,2021-06-01,59.0,Above average
3,621e375b67b776a240290cdc,2021-06-22,60.0,Above average


In [10]:
fitbit_stai = preprocessing_functions.weekly_fitbit_frequency(stai, fitbit, users)
fitbit_stai

,id,date,stai_stress,stai_stress_category,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,...,Martial Arts,Run,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates
0,621e2e8e67b776a24055b564,2021-05-31,45.0,Below average,34.073346,51.364,51.324,96.1,12.2,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,621e2e8e67b776a24055b564,2021-06-07,46.0,Average,34.073346,51.364,51.324,96.1,12.2,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,621e2e8e67b776a24055b564,2021-07-19,46.0,Average,34.073346,51.364,51.324,96.1,12.2,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,621e2e8e67b776a24055b564,2021-07-26,46.0,Average,34.073346,51.364,51.324,96.1,12.2,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,621e2e8e67b776a24055b564,2021-06-28,54.0,Above average,34.073346,51.364,51.324,96.1,12.2,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,621e339967b776a240e502de,2021-11-22,49.0,Average,34.073346,51.364,51.324,96.1,12.2,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
275,621e339967b776a240e502de,2021-12-06,51.0,Above average,34.073346,51.364,51.324,96.1,12.2,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
276,621e339967b776a240e502de,2021-12-14,45.0,Below average,34.073346,51.364,51.324,96.1,12.2,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
277,621e339967b776a240e502de,2021-12-20,48.0,Average,34.073346,51.364,51.324,96.1,12.2,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [11]:
fitbit_stai.to_pickle('./data/fitbit_stai_to_model.pkl')